In [5]:
#IMPORT STATEMENTS
import pandas as pd

In [ ]:
#read by chunks
#for chunk in pd.read_csv("2011.csv", chunksize=10):
    #print(chunk)

In [6]:
#read large dataset structure
import dask.dataframe as dd

In [7]:
df_shape = dd.read_csv('2011.csv')

In [8]:
df_shape

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,object,object,object,int64,object,object,float64,object,object,object,object,int64,int64,int64,object,object,object,bool,float64,int64,float64,float64,float64,float64,float64,float64,object,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [13]:
#filter out by needed columns
df = pd.read_csv('2011.csv', usecols=['date', 'tweet', 'hashtags'])
df

,date,tweet,hashtags
0,01/01/2011,Check out @CDCFlu pledge page for National Inf...,['#cdcnivw']
1,01/01/2011,In your go-kit should be your pets vaccination...,[]
2,01/01/2011,Vaccination are legal poison. Try to avoid the...,[]
3,01/01/2011,Have you found ways to detox after a vaccinati...,[]
4,01/01/2011,AMESSI - La grippe A(H1N1) relance le débat su...,['#h1n1']
...,...,...,...
117590,31/12/2011,FDA clears Pfizer vaccine for adults over 50 -...,"['#vaccination', '#sf']"
117591,31/12/2011,More parents pass on shots - Massillon Indepen...,"['#vaccination', '#sf']"
117592,31/12/2011,Pet health alert: Rabies vaccine mandatory in ...,"['#vaccination', '#sf']"
117593,31/12/2011,CDC expands access to public flu vaccine - Tai...,"['#vaccination', '#sf']"


In [10]:
#check for null values
df.isnull().sum()

date        0
tweet       0
hashtags    0
dtype: int64

In [16]:
#dropping duplicate rows
#analysis:
print(len(df.index)) #117595 total tweets
df_dup = df.duplicated().tolist()
print(df_dup.count(True)) #9512 duplicated user tweets
df_dup = df.duplicated(['tweet']).tolist()
print(df_dup.count(True)) #17999 duplicated full texts

117595
9512
17999


In [18]:
df = df.drop_duplicates(['tweet']) #drop duplicate tweets
df

,date,tweet,hashtags
0,01/01/2011,Check out @CDCFlu pledge page for National Inf...,['#cdcnivw']
1,01/01/2011,In your go-kit should be your pets vaccination...,[]
2,01/01/2011,Vaccination are legal poison. Try to avoid the...,[]
3,01/01/2011,Have you found ways to detox after a vaccinati...,[]
4,01/01/2011,AMESSI - La grippe A(H1N1) relance le débat su...,['#h1n1']
...,...,...,...
117590,31/12/2011,FDA clears Pfizer vaccine for adults over 50 -...,"['#vaccination', '#sf']"
117591,31/12/2011,More parents pass on shots - Massillon Indepen...,"['#vaccination', '#sf']"
117592,31/12/2011,Pet health alert: Rabies vaccine mandatory in ...,"['#vaccination', '#sf']"
117593,31/12/2011,CDC expands access to public flu vaccine - Tai...,"['#vaccination', '#sf']"


In [7]:
# CLEANUP COMPILER
import re

In [2]:
#cleaning out text:

def cleanTweet(tweet):
    tweet.strip()
    tweet = re.sub(r'@\w+: ?', '', tweet) #cleanout mentions:
    tweet = re.sub(r'@\w* ?', '', tweet) #clean out mentions
    tweet = re.sub(r'#\w+ ?', '', tweet) #remove hashtags symbol
    tweet = re.sub(r'https?:\/\/\S+', '', tweet) #remove hyperlinks
    tweet = re.sub(r'(?<![\w\-])\w(?![\w\-])', '', tweet) #remove single word patterns
    tweet = re.sub(r'\s{2,}|\t', '', tweet) #remove blank spaces pattern
    tweet = re.sub(r'(RT|rt|FAV|fav|VIA|via)', '', tweet) #remove reserved words
    return tweet

# Decision not to deEmojify as vaderSentiment works well with emojis
# def deEmojify(tweet):
#     emojis = re.compile(pattern = "["
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                            "]+", flags = re.UNICODE)
#     return emojis.sub(r'',tweet)

In [9]:
#make new column for cleaned tweets
df['clean_tweets'] = df['tweet'].apply(cleanTweet)

In [180]:
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

ModuleNotFoundError: No module named 'vaderSentiment'

In [15]:
#negative sentiment scores
def sentiment_analyzer_scores_neg(tweet):
    score = analyser.polarity_scores(tweet)
    return score['neg']

In [16]:
#neutral sentiment scores
def sentiment_analyzer_scores_neu(tweet):
    score = analyser.polarity_scores(tweet)
    return score['neu']

In [17]:
#positive sentiment scores
def sentiment_analyzer_scores_pos(tweet):
    score = analyser.polarity_scores(tweet)
    return score['pos']

In [18]:
#compound sentiment scores
def sentiment_analyzer_scores_compound(tweet):
    score = analyser.polarity_scores(tweet)
    return score['compound']

In [19]:
#apply analyzers to columns
df['neg'] = df['clean_tweets'].apply(sentiment_analyzer_scores_neg)

NameError: name 'analyser' is not defined

In [270]:
df['neu'] = df['clean_tweets'].apply(sentiment_analyzer_scores_neu)

In [188]:
df['pos'] = df['clean_tweets'].apply(sentiment_analyzer_scores_pos)

In [189]:
df['compound'] = df['clean_tweets'].apply(sentiment_analyzer_scores_compound)

In [193]:
#calculate expanding mean
df['mean'] = df['compound'].expanding().mean()

#calculate rolling mean
df['rolling'] = df['compound'].rolling(1).mean()

In [299]:
df.head(30)

,date,tweet,hashtags,clean_tweets,neg,neu,pos,compound,mean,rolling
0,01/01/2011,Check out @CDCFlu pledge page for National Inf...,['#cdcnivw'],Check out pledge page for National Influenza V...,0.000,1.000,0.000,0.0000,0.000000,0.0000
2,01/01/2011,Vaccination are legal poison. Try to avoid the...,[],Vaccination are legal poison. Try to avoid the...,0.375,0.526,0.099,-0.6369,-0.212300,-0.6369
3,01/01/2011,Have you found ways to detox after a vaccinati...,[],Have you found ways to detox aftervaccination?,0.000,1.000,0.000,0.0000,-0.159225,0.0000
4,01/01/2011,AMESSI - La grippe A(H1N1) relance le débat su...,['#h1n1'],AMESSI - La grippe (H1N1) relance le débat sur...,0.000,1.000,0.000,0.0000,-0.127380,0.0000
5,01/01/2011,RT @daffynitions: Vaccination: the medical sac...,[],Vaccination: the medical sacrament correspond...,0.000,1.000,0.000,0.0000,-0.106150,0.0000
6,01/01/2011,Children With Autism Have Mitochondrial Dysfun...,[],Children With Autism Have Mitochondrial Dysfun...,0.157,0.843,0.000,-0.4215,-0.151200,-0.4215
9,01/01/2011,@jonbecker she is the anti-vaccination zealot ...,[],she is the anti-vaccination zealot obviously.,0.367,0.633,0.000,-0.4404,-0.149880,-0.4404
10,01/01/2011,Russian activist disappears after exposing va...,[],Russian activist disappears after exposingvacc...,0.167,0.833,0.000,-0.5423,-0.185555,-0.5423
11,01/01/2011,@Dyslexic_Trojan Yes it is but the people that...,[],Yes it is but the people that are being affect...,0.301,0.631,0.066,-0.7391,-0.231683,-0.7391
12,01/01/2011,HSE renews advice to get flu vaccination http...,[],HSE renews advice to get flu vaccination,0.302,0.698,0.000,-0.3818,-0.243231,-0.3818
